In [1]:
import random
import numpy as np
import torch
import json
import os
from tqdm import tqdm
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import copy

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

ROOT = Path("Amazon_products")
TRAIN_DIR = ROOT / "train"
TEST_DIR = ROOT / "test"

TEST_CORPUS_PATH = TEST_DIR / "test_corpus.txt"
TRAIN_CORPUS_PATH = TRAIN_DIR / "train_corpus.txt"

CLASS_HIERARCHY_PATH = ROOT / "class_hierarchy.txt"
CLASS_RELATED_PATH = ROOT / "class_related_keywords.txt"
CLASS_PATH = ROOT / "classes.txt"

SUBMISSION_PATH = "Submission/submission.csv"

NUM_CLASSES = 531
MIN_LABELS = 2
MAX_LABELS = 3

def load_corpus(path):
    """Load corpus into {id: text} dictionary."""
    id2text = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t", 1)
            if len(parts) == 2:
                id, text = parts
                id2text[id] = text
    return id2text

def load_multilabel(path):
    """Load multi-label data into {id: [labels]} dictionary."""
    id2labels = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) == 2:
                pid, label = parts
                pid = int(pid)
                label = int(label)
                if pid not in id2labels:
                    id2labels[pid] = []
                id2labels[pid].append(label)
    return id2labels

def load_class_keywords(path):
    """Load class keywords into {class_name: [keywords]} dictionary."""
    class2keywords = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" not in line:
                continue
            classname, keywords = line.strip().split(":", 1)
            keyword_list = [kw.strip() for kw in keywords.split(",") if kw.strip()]
            class2keywords[classname] = keyword_list
    return class2keywords

id2text_test = load_corpus(TEST_CORPUS_PATH)
id2text_train = load_corpus(TRAIN_CORPUS_PATH)

# Classes
id2class = load_corpus(CLASS_PATH)
class2hierarchy = load_multilabel(CLASS_HIERARCHY_PATH)
class2related = load_class_keywords(CLASS_RELATED_PATH)


print(f"Train: {len(id2text_train)} samples")
print(f"Classes: {len(id2class)}")

# Embeddings
X_train = torch.load("Embeddings/X_train.pt").to(device)
X_test = torch.load("Embeddings/X_test.pt").to(device)
label_emb = torch.load("Embeddings/label_emb.pt").to(device)
test_ids = list(id2text_test.keys())
train_ids = list(id2text_train.keys())

print(f"Train embeddings: {X_train.shape}")
print(f"Test embeddings: {X_test.shape}")
print(f"Label embeddings: {label_emb.shape}")

# Index mapping
pid2idx = {pid: i for i, pid in enumerate(train_ids)}

input_dim = X_train.size(1)
num_classes = NUM_CLASSES

print(f"Input dimension: {input_dim}")
print(f"Num classes: {num_classes}")


with open("Silver/silver_fixed.json", "r") as f:
    raw_silver = json.load(f)

if "silver_labels" in raw_silver:
    silver_data = raw_silver["silver_labels"]
else:
    silver_data = raw_silver  # fallback si le fichier ne contient que les labels

pid2labelids_silver = {
    int(pid): info["labels"]
    for pid, info in silver_data.items()
    if "labels" in info
}



Device: cuda
Train: 29487 samples
Classes: 531
Train embeddings: torch.Size([29487, 768])
Test embeddings: torch.Size([19658, 768])
Label embeddings: torch.Size([531, 768])
Input dimension: 768
Num classes: 531


In [2]:
# Adjency matrix A
A = torch.zeros((num_classes, num_classes), dtype=torch.float32, device=device)

for parent_id, children_ids in class2hierarchy.items():
    if parent_id >= num_classes:
        continue
    for child_id in children_ids:
        if child_id >= num_classes:
            continue
        # Bidirectional link
        A[parent_id, child_id] = 1.0
        A[child_id, parent_id] = 1.0

A = A + torch.eye(num_classes, device=device)

# D
deg = A.sum(dim=1) 
D = torch.diag(deg)

print(f"Adjacency matrix A: {A.shape}")
print(f"Degree matrix D: {D.shape}")

# Normalisation : A_hat = D^{-1/2} @ A @ D^{-1/2}
deg_inv_sqrt = torch.pow(deg, -0.5)
deg_inv_sqrt[torch.isinf(deg_inv_sqrt)] = 0.0
D_inv_sqrt = torch.diag(deg_inv_sqrt)

A_hat = D_inv_sqrt @ A @ D_inv_sqrt
print(f"Normalized adjacency A_hat: {A_hat.shape}")

Adjacency matrix A: torch.Size([531, 531])
Degree matrix D: torch.Size([531, 531])
Normalized adjacency A_hat: torch.Size([531, 531])


In [3]:
class LabelGCN(nn.Module):
    """
    Multi-layer Graph Convolutional Network (GCN) encoder for label embeddings.

    Each layer should perform the following steps:
        1. Aggregate neighbor embeddings: H <- A_hat @ H
        2. Linear transformation: H <- H @ W
        3. (Optional) Apply ReLU and Dropout (skip for the last layer)

    Args:
        emb_dim (int): Dimension of label embeddings.
        num_layers (int): Number of GCN layers.
        dropout (float): Dropout probability.
    """
    def __init__(self, emb_dim, num_layers=1, dropout=0.5):
        super().__init__()
        self.emb_dim = emb_dim
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.W_list = nn.ParameterList()
        for _ in range(num_layers):
            W = nn.Parameter(torch.empty(emb_dim, emb_dim))
            nn.init.xavier_uniform_(W)
            self.W_list.append(W)

    def forward(self, H, A_hat):
        """
        Args:
            H (torch.Tensor): Initial label embeddings, shape (num_labels, emb_dim).
            A_hat (torch.Tensor): Normalized adjacency matrix, shape (num_labels, num_labels).

        Returns:
            torch.Tensor: Updated label embeddings, shape (num_labels, emb_dim).
        """
        for i, W in enumerate(self.W_list):
            H = A_hat @ H @ W
            
            if i < self.num_layers - 1:
                H = F.relu(H)
                H = F.dropout(H, p=self.dropout, training=self.training)
        
        return H


class GCNEnhancedClassifier(nn.Module):
    """
    Classifier that combines:
      - Document representations projected into label space
      - Label embeddings refined by a GCN over the label graph

    Args:
        input_dim (int): Dimension of input document embeddings.
        label_init_emb (torch.Tensor): Initial label embeddings, shape (num_labels, emb_dim).
        A_hat (torch.Tensor): Normalized adjacency matrix of labels, shape (num_labels, num_labels).
        num_layers (int): Number of GCN layers.
        dropout (float): Dropout probability.
    """
    def __init__(self, input_dim, label_init_emb, A_hat, num_layers=1, dropout=0.5):
        super().__init__()
        self.proj = nn.Linear(input_dim, label_init_emb.size(1)) #1

        self.dropout = dropout
        self.num_layers = num_layers
        self.encoder = LabelGCN(label_init_emb.size(1), num_layers, dropout) #2

        self.label_emb = nn.Parameter(label_init_emb.clone()) #3

        self.register_buffer("A_hat", A_hat) #4

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input embeddings for documents, shape (batch_size, input_dim).

        Returns:
            torch.Tensor: Logits for classification, shape (batch_size, num_labels).
        """
        E_refine = self.encoder(self.label_emb, self.A_hat) #1

        x_proj = self.proj(x)
        x_proj = F.dropout(x_proj, p=self.dropout, training=self.training) #2

        logits = torch.matmul(x_proj, E_refine.T) #3
        return logits

In [4]:
class ProductCategoryDataset(Dataset):
    """Dataset using pre-calculated embeddings (train or test compatible)"""
    def __init__(self, pid2label, pid2idx, embeddings, num_classes=531):
        self.pid2label = pid2label
        self.pid2idx = pid2idx
        self.embeddings = embeddings
        self.num_classes = num_classes

        if self.pid2label is not None:
            self.pids = list(pid2label.keys())
            self.has_labels = True
        else:
            self.pids = list(pid2idx.keys())
            self.has_labels = False

        self.indices = [pid2idx[pid] for pid in self.pids]

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        emb = self.embeddings[self.indices[idx]]

        if self.has_labels:
            y = torch.zeros(self.num_classes, dtype=torch.float32)
            for label in self.pid2label[self.pids[idx]]:
                y[label] = 1.0
            return {"X": emb, "y": y}
        else:
            return {"X": emb}


In [13]:
import torch
import json
from sklearn.model_selection import train_test_split

# === Load mappings ===
pid2idx = torch.load("Embeddings/pid2idx.pt")  # ✅ recharge le bon mapping

# === Load Silver Labels ===
with open("Silver/silver_fixed.json", "r") as f:
    raw_silver = json.load(f)

pid2labelids_silver = {
    int(pid): info["labels"]
    for pid, info in raw_silver["silver_labels"].items()
}

# === Split train/val ===
silver_pids = list(pid2labelids_silver.keys())
train_pids, val_pids = train_test_split(silver_pids, test_size=0.2, random_state=42)

train_labels = {pid: pid2labelids_silver[pid] for pid in train_pids}
val_labels = {pid: pid2labelids_silver[pid] for pid in val_pids}

print(f"Train: {len(train_labels)} | Val: {len(val_labels)}")

# === Filter valid ===
valid_pids = set(pid2idx.keys())
train_labels = {pid: labels for pid, labels in train_labels.items() if pid in valid_pids}
val_labels = {pid: labels for pid, labels in val_labels.items() if pid in valid_pids}

print(f"✓ Train valid: {len(train_labels)} | Val valid: {len(val_labels)}")
missing = [pid for pid in silver_pids if pid not in pid2idx]
print(f"⚠️ Missing {len(missing)} PIDs (ex: {missing[:5]})")

# === Dataset & Loaders ===
train_dataset = ProductCategoryDataset(train_labels, pid2idx, X_train, num_classes=NUM_CLASSES)
val_dataset = ProductCategoryDataset(val_labels, pid2idx, X_train, num_classes=NUM_CLASSES)

test_dataset = ProductCategoryDataset(
    None,
    {pid: i for i, pid in enumerate(test_ids)},
    X_test,
    num_classes=NUM_CLASSES
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

Train: 23589 | Val: 5898
✓ Train valid: 23589 | Val valid: 5898
⚠️ Missing 0 PIDs (ex: [])


In [14]:
def compute_example_f1(y_true, y_pred):
    """
    Example-F1 (Samples F1-score)
    
    Formula: Example-F1 = (1/N) × Σ [2×|Ctrue ∩ Cpred| / (|Ctrue| + |Cpred|)]
    
    Args:
        y_true: array (n_samples, n_classes) - labels vrais (0 ou 1)
        y_pred: array (n_samples, n_classes) - labels prédits (0 ou 1)
    
    Returns:
        float: Example-F1 score entre 0 et 1
    """
    f1_per_sample = []
    
    for i in range(len(y_true)):
        true_set = set(np.where(y_true[i] == 1)[0])
        pred_set = set(np.where(y_pred[i] == 1)[0])
        
        # Cas spéciaux
        if len(true_set) == 0 and len(pred_set) == 0:
            f1_per_sample.append(1.0)
        elif len(true_set) == 0 or len(pred_set) == 0:
            f1_per_sample.append(0.0)
        else:
            intersection = len(true_set & pred_set)
            f1 = 2 * intersection / (len(true_set) + len(pred_set))
            f1_per_sample.append(f1)
    
    return np.mean(f1_per_sample)

def evaluate_complete(model, dataloader, device="cpu", threshold=0.5):
    """
    Standard evaluation function for multi-label classification.

    Computes:
      - example_f1 : average F1 per sample
      - f1_macro   : unweighted class average
      - f1_micro   : global average across all decisions
      - f1_weighted: sample-weighted (by class frequency)
      - exact_match: proportion of samples with all labels correct
    """
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y = batch["y"].cpu().numpy()

            logits = model(X)
            probs = torch.sigmoid(logits).cpu().numpy()
            preds = (probs > threshold).astype(int)

            all_preds.append(preds)
            all_labels.append(y)

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    example_f1 = compute_example_f1(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)
    f1_micro = f1_score(all_labels, all_preds, average="micro", zero_division=0)
    f1_samples = f1_score(all_labels, all_preds, average="samples", zero_division=0)

    return {
        "example_f1": example_f1,
        "sample_f1": f1_samples,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro
    }


In [15]:
# Init
model = GCNEnhancedClassifier(
    input_dim=input_dim,
    label_init_emb=label_emb,
    A_hat=A_hat,
    num_layers=2,
    dropout=0.3
).to(device)

# Teacher model (EMA)
teacher = copy.deepcopy(model).to(device)
for param in teacher.parameters():
    param.requires_grad = False

# Optimizer & Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
criterion = nn.BCEWithLogitsLoss()

print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")

Total params: 2,178,048


In [16]:
# Configuration
alpha_ema = 0.995
lambda_cons = 0.5
best_val_f1 = -1
best_model_state = None
patience = 5
patience_counter = 0
EPOCHS = 25

# Tracking
results_dict = {'valid': []}

for epoch in range(1, EPOCHS + 1):
    print(f"\n=== Epoch {epoch}/{EPOCHS} ===")
    
    model.train()
    teacher.train()
    
    total_loss = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        X = batch["X"].to(device)
        y = batch["y"].to(device)
        
        # Student predictions
        logits_student = model(X)
        loss_sup = criterion(logits_student, y)
        
        # Teacher predictions
        with torch.no_grad():
            logits_teacher = teacher(X)
        
        # Consistency loss
        loss_cons = F.mse_loss(torch.sigmoid(logits_student), torch.sigmoid(logits_teacher))
        
        # Total loss
        loss = loss_sup + lambda_cons * loss_cons
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Update teacher (EMA)
        with torch.no_grad():
            for param_teacher, param_student in zip(teacher.parameters(), model.parameters()):
                param_teacher.data = alpha_ema * param_teacher.data + (1 - alpha_ema) * param_student.data
        
        total_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
    
    print(f"Loss: {total_loss / len(train_loader):.4f}")
    
    # Validation
    val_metrics = evaluate_complete(teacher, val_loader, device=device)
    val_f1 = val_metrics['example_f1']
    f1_test = val_metrics['sample_f1']
    results_dict['valid'].append(val_f1)
    print(f"Val F1: {val_f1:.4f}")
    print(f"Sample F1: {f1_test:.4f}")
    
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model_state = copy.deepcopy(teacher.state_dict())
        patience_counter = 0
        print(f"New best model")
    else:
        patience_counter += 1
    
    # Early stopping
    if patience_counter >= patience:
        print(f"\nEarly stopping")
        break
    
    scheduler.step()

print(f"\n{'='*50}")
print(f"Best Val F1: {best_val_f1:.4f}")
print(f"{'='*50}\n")



=== Epoch 1/25 ===


Training: 100%|██████████| 369/369 [00:03<00:00, 116.39it/s, loss=0.0111]


Loss: 0.0688
Val F1: 0.0000
Sample F1: 0.0000
New best model

=== Epoch 2/25 ===


Training: 100%|██████████| 369/369 [00:02<00:00, 131.93it/s, loss=0.0097]


Loss: 0.0107
Val F1: 0.6222
Sample F1: 0.6222
New best model

=== Epoch 3/25 ===


...

Loss: 0.0078
Val F1: 0.6668
Sample F1: 0.6668

=== Epoch 24/25 ===


Training: 100%|██████████| 369/369 [00:02<00:00, 132.60it/s, loss=0.0077]


Loss: 0.0077
Val F1: 0.6679
Sample F1: 0.6679
New best model

=== Epoch 25/25 ===


Training: 100%|██████████| 369/369 [00:02<00:00, 136.09it/s, loss=0.0075]


Loss: 0.0078
Val F1: 0.6685
Sample F1: 0.6685
New best model

Best Val F1: 0.6685



In [17]:
# Charger le meilleur état du modèle enseignant
teacher.load_state_dict(best_model_state)
teacher.eval()

all_preds = []
teacher.eval()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting on test"):
        X = batch["X"].to(device)
        logits = teacher(X)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_preds.append(probs)

# Fusionner les prédictions
all_preds = np.vstack(all_preds)

threshold = 0.5
predicted_labels = [np.where(row > threshold)[0].tolist() for row in all_preds]

import pandas as pd

submission_rows = []
for pid, labels in zip(test_ids, predicted_labels):
    labels_str = " ".join(map(str, labels))
    submission_rows.append({"id": pid, "label": labels_str})

df_submission = pd.DataFrame(submission_rows)
df_submission.to_csv(SUBMISSION_PATH, index=False)
print(f"✅ Submission saved to {SUBMISSION_PATH}")

Predicting on test: 100%|██████████| 308/308 [00:00<00:00, 522.49it/s]


✅ Submission saved to Submission/submission.csv


In [ ]:
import csv, os
from tqdm import tqdm
import torch
import numpy as np

X_test = torch.load("Embeddings/X_test.pt").to(device)

THRESHOLD = 0.5
MIN_LABELS = 2      # 👈 au moins 2 labels
MAX_LABELS = 3      # 👈 au maximum 3 labels
OUTPUT_PATH = "Submission/GNN.csv"
os.makedirs("Submission", exist_ok=True)

all_pids, all_pred_labels = [], []

with torch.no_grad():
    for start in tqdm(range(0, len(X_test), 64), desc="Generating predictions"):
        end = start + 64
        batch = X_test[start:end]
        batch_pids = test_ids[start:end]

        logits = teacher(batch)
        probs = torch.sigmoid(logits).cpu().numpy()

        for pid, prob in zip(batch_pids, probs):
            # prédiction binaire selon le seuil
            pred_row = (prob > THRESHOLD).astype(int)

            # --- ⚙️ Aucune prédiction : on prend le top-2 (minimum garanti)
            if pred_row.sum() == 0:
                topk_idx = np.argsort(prob)[-MIN_LABELS:][::-1]
                pred_row = np.zeros_like(pred_row)
                pred_row[topk_idx] = 1

            # --- ⚙️ Seulement 1 label : on ajoute le deuxième plus probable
            elif pred_row.sum() == 1:
                top2_idx = np.argsort(prob)[-2:][::-1]
                pred_row = np.zeros_like(pred_row)
                pred_row[top2_idx] = 1

            # --- ⚙️ Trop de labels : on garde seulement les top-3
            elif pred_row.sum() > MAX_LABELS:
                topk_idx = np.argsort(prob)[-MAX_LABELS:][::-1]
                pred_row = np.zeros_like(pred_row)
                pred_row[topk_idx] = 1

            # --- Labels finaux
            labels = [str(j) for j, v in enumerate(pred_row) if v == 1]

            all_pids.append(pid)
            all_pred_labels.append(labels)

print(f"{len(all_pids)} samples generated.")

# --- Sauvegarde CSV
with open(OUTPUT_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "label"])
    for pid, labels in zip(all_pids, all_pred_labels):
        writer.writerow([pid, ",".join(labels)])

print(f"✅ Submission file saved: {OUTPUT_PATH}")


Generating predictions: 100%|██████████| 308/308 [00:01<00:00, 181.95it/s]

19658 samples generated.
✅ Submission file saved: Submission/GNN.csv
